# Machine Learning with Python

In [2]:
import numpy as np
import itertools
import json

In [5]:
def create_matrix(order):
    def create_keys(order):
        keys = ['R', 'P', 'S']
        for i in range((order * 2 - 1)):
            key_len = len(keys)
            for i in itertools.product(keys, ''.join(keys)):
                keys.append(''.join(i))
            keys = keys[key_len:]
        return keys
    
    keys = create_keys(order)

    matrix = {}
    for key in keys:
        matrix[key] = {
            'R': {
                'prob': 1/3,
                'n_obs': 0
            },
            'P': {
                'prob': 1/3,
                'n_obs': 0
            },
            'S': {
                'prob': 1/3,
                'n_obs': 0
            }
        }
    
    return matrix

In [6]:
m = create_matrix(1)
print(json.dumps(m, indent = 4))

{
    "RR": {
        "R": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "P": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "S": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        }
    },
    "RP": {
        "R": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "P": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "S": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        }
    },
    "RS": {
        "R": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "P": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "S": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        }
    },
    "PR": {
        "R": {
            "prob": 0.3333333333333333,
            "n_obs": 0
        },
        "P": {
            "prob": 0.33333

---

# Project 1: Rock Paper Scissors
Create a bot that beats the other bits, at least 60% of the games.

## Method
Markov Chain

## What is a Markov Chain
> A stochastic process containing random variables, transitioning from one state to another depending on certain assumptions and definite probabilistic rules.

In [6]:
import numpy as np
from collections import Counter

In [3]:
T = np.ones((3, 3))
T = T * 1/3

In [4]:
RPS = ['R', 'P', 'S']

In [20]:
# mock up of opponents moves
OM = ['R', 'R', 'S', 'R', 'P', 'S', 'S', 'S', 'P', 'P', 'R', 'P', 'S', 'R']
# convert to ints
RPS_TO_INT = {
    'R': 0,
    'P': 1,
    'S': 2
}
OM_INT = []
for rps in OM:
    OM_INT.append(RPS_TO_INT[rps])

print(OM_INT)
# Incase going back to RPS is needed:
INT_TO_RPS = {v: k for k, v in RPS_TO_INT.items()}

# testing it
INT_OM = []
for i in OM_INT:
    INT_OM.append(INT_TO_RPS[i])

print(INT_OM)

[0, 0, 2, 0, 1, 2, 2, 2, 1, 1, 0, 1, 2, 0]
['R', 'R', 'S', 'R', 'P', 'S', 'S', 'S', 'P', 'P', 'R', 'P', 'S', 'R']


In [21]:
# Calculate average opponent moves
# First, count the occurances
C = Counter(OM)
print(C)

Counter({'R': 5, 'S': 5, 'P': 4})


In [12]:
# Rephrasing: what is the probability of the opponents next move? So, what is the current state and what will then be the opponents action?
# Continously modify the transition matrix to adapt to the new opponent data.
# Transistion matrix: What is the probability of taking a certain action, resulting in a next state, depending on the current state?

# Get the mock up opponent data and turn it into a transistion matrix

In [13]:
def transition_matrix(t):
    M = np.zeros((3, 3))
    for (i, j) in zip(t, t[1:]):
        M[i][j] += 1
    
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]

    return M

In [15]:
m = transition_matrix(OM_INT)
print(m)

[[0.33333333 0.33333333 0.33333333]
 [0.         0.         0.        ]
 [1.         0.         0.        ]]


In [16]:
def t_m(t, m):
    for (i, j) in zip(t, t[1:]):
        m[i, j] += 1

    for row in m:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    
    return m

In [24]:
mm = np.array(t_m(OM_INT, T))
print(mm)

[[0.25066667 0.49866667 0.25066667]
 [0.24933333 0.24933333 0.50133333]
 [0.39969136 0.20015432 0.40015432]]


In [32]:
# so, if the previous state was Scissors, the new state ought to be Scissors as well. I.E:
next_state = INT_TO_RPS[np.argmax(mm[RPS_TO_INT['S']])]
print(next_state)

S


In [34]:
# This should lead to the following guess
best_guess = {
    'R': 'P',
    'P': 'S',
    'S': 'R'
}

guess = best_guess[next_state]
print(guess)

R


### How to introduce the Bellman equation before making the guess?
We now have the next state, i.e. the next move that the opponent will make. We have the current state, which in our case is the prev_play in the original code. What then, is the action? Is it the action we're going to take, i.e. our guess?

---

I have no idea how to make MDP and bellman work by beating the bots at least 60%

---

In [8]:
mymoves = ['R', 'R', 'S', 'R', 'S', 'P', 'P', 'S', 'R', 'S', 'R', 'R', 'R']
myLastTen = mymoves[-10:]
mymostFrequentMove = max(set(myLastTen), key=myLastTen.count)
print(myLastTen)
print(set(myLastTen))
print(mymostFrequentMove)

['R', 'S', 'P', 'P', 'S', 'R', 'S', 'R', 'R', 'R']
{'R', 'P', 'S'}
R


In [17]:
strategy = [0, 0, 0, 0]
opponent_guess = ['', '', '', '']
prev_play = 'R'
opponent_history = ['R', 'R', 'S', 'P', 'S', 'S', 'S', 'R', 'P']
for i in range(0, 4):
    if opponent_guess[i] == prev_play:
        strategy[i] += 1

print(strategy)

[0, 0, 0, 0]


In [10]:
def predict_move(history, n, play_order):
    if "".join(history[-n:]) in play_order.keys():
        play_order["".join(history[-n:])] += 1
    else:
        play_order["".join(history[-n:])] = 1
    possible = ["".join(history[-(n - 1) :]) + k for k in ["R", "P", "S"]]
    for pm in possible:
        if not pm in play_order.keys():
            play_order[pm] = 0
    predict = max(possible, key=lambda key: play_order[key])
    return predict[-1]

In [14]:
opponent_play_order = {}

In [15]:
predict_move(opponent_history, 3, opponent_play_order)

'S'

In [18]:
anOrder = {}
anOrder[''.join(opponent_history[-3:])] = 1
print(anOrder)

{'SRP': 1}


In [21]:
possible = [''.join(opponent_history[-(3-1):]) + k for k in ['R', 'P', 'S']]
print(possible)

['RPR', 'RPP', 'RPS']


In [22]:
''.join(opponent_history[-(3-1):])

'RP'

In [24]:
m = ['' + k for k in ['R', 'P', 'S']]
print(m)

['R', 'P', 'S']


In [25]:
opponent_play_order

{'SSS': 1, 'SSR': 0, 'SSP': 0}

In [26]:
for pm in possible:
    if not pm in opponent_play_order.keys():
        opponent_play_order[pm] = 0
print(opponent_play_order)

{'SSS': 1, 'SSR': 0, 'SSP': 0, 'RPR': 0, 'RPP': 0, 'RPS': 0}


In [27]:
predict = max(possible, key = lambda key: opponent_play_order[key])
print(predict)

RPR


In [28]:
opponent_play_order.keys()

dict_keys(['SSS', 'SSR', 'SSP', 'RPR', 'RPP', 'RPS'])